In [1]:
import os
from docx import Document
import chromadb
from chromadb.config import Settings
from openai import OpenAI
import pandas as pd
import PyPDF2
import os
import json
import re
from sentence_transformers import SentenceTransformer, util
from dotenv import load_dotenv
load_dotenv()


global collection,status_msg
status_msg = {}
# --------------------------
# 1. Load and Chunk Document
# --------------------------

# defining path constants
output_folder = 'expert/extracted_controls'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def clean_text(text):
    lines = text.split("\n")

    control_blocks = []
    current_block = []

    # Regex for ID (numbers only OR A.12.3)
    control_id_pattern = re.compile(r'\b(?:[A-Z]+(?:\.[A-Z]+)*-?)?\d+(?:\.\d+)*\b')

    for line in lines:
        stripped = line.strip()
        if not stripped:
            continue

        # Detect start of a new control
        if control_id_pattern.match(stripped):
            if current_block:
                control_blocks.append("\n".join(current_block))
            current_block = [stripped]
        else:
            if current_block:
                current_block.append(stripped)

    if current_block:
        control_blocks.append("\n".join(current_block))

    final_blocks = []

    for block in control_blocks:
        block_lines = block.strip().split("\n")

        # -------------------------
        # 1. CHECK ID
        # -------------------------
        first_line = block_lines[0]
        if not control_id_pattern.match(first_line):
            continue

        # -------------------------
        # 2. CHECK NAME (Title)
        # It can be on SAME line OR NEXT line
        # -------------------------

        name_found = False

        # (A) Try same line (after removing ID)
        first_line_without_id = control_id_pattern.sub("", first_line).strip()
        if re.search(r'[A-Za-z]', first_line_without_id):
            name_found = True

        # (B) Try next line if no name found yet
        if not name_found and len(block_lines) > 1:
            second_line = block_lines[1].strip()
            # Next line should contain alphabetic words (name/title)
            if re.search(r'[A-Za-z]', second_line) and len(second_line.split()) <= 10:
                # Names are usually short (<=10 words)
                name_found = True

        if not name_found:
            continue    # ❌ No title → discard entire block

        # -------------------------
        # 3. CHECK DESCRIPTION
        # Description must contain ≥5 words (after removing ID & name)
        # -------------------------
        # Description starts from either 2nd or 3rd line
        desc_start_index = 1 if first_line_without_id else 2

        description_lines = block_lines[desc_start_index:]
        description_text = " ".join(description_lines).strip()

        # Remove pure titles from description
        # Description must have meaningful text
        if len(description_text.split()) <2:
            continue  # ❌ Not enough meaningful info

        # All 3 components exist → keep block
        final_blocks.append(block)

    return "\n\n".join(final_blocks)


def normalize_docx_spacing(text):
    # Remove spacing between single characters
    # "t o   P I I" → "to PII"
    text = re.sub(r"(?<=\w)\s+(?=\w)", " ", text)

    # Remove double or triple spaces
    text = re.sub(r"\s{2,}", " ", text)

    return text


#The following function returns output in list form with cleaning of text

#The following function returns output in String form
def load_docx(path):
    doc = Document(path)
    lines = []

    for para in doc.paragraphs:
        text = para.text.strip()
        if text:
            lines.append(text)

    # Return as a single string (joined by newline)

    return normalize_docx_spacing("\n".join(lines))


def load_pdf_text(file_path):
    """
    Loads and extracts text from a PDF file.
    Returns the extracted text in the form of a list.
    """
    text = ""

    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)

        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text() + "\n"
            text=clean_text(text)

    return text.split("\n")

def load_txt(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read().split("\n")


import pandas as pd

def load_excel(file_path):
    xls = pd.ExcelFile(file_path)
    text = ""

    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
        text += df.to_string(index=False) + "\n\n"

    return text.split("\n")



def load_document(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    print(ext)

    if ext == ".docx":
        return load_docx(file_path)
    elif ext == ".pdf":
        return load_pdf_text(file_path)
    elif ext == ".txt":
        return load_txt(file_path)
    elif ext in [".xls", ".xlsx"]:
        return load_excel(file_path)
    else:
        raise ValueError(f"Unsupported file type: {ext}")


#Extracting controls from the document of Client


#The following function removes bullets, dots and numbering
def clean_leading_bullets(text):
    if not text:
        return text

    # remove typical bullet characters
    text = re.sub(r"^[\s•·▪○●►▶↳\-\*]+", "", text)

    # remove stray dots like ". " or " . "
    text = re.sub(r"^\s*\.\s*", "", text)
    """
    Removes leading numbering patterns like:
    '8. Text', '8) Text', '(8) Text', '8 - Text', '8: Text'
    """
    return re.sub(r"^\s*\(?\d+\)?\s*[\.\:\-\)]\s*", "", text).strip()

    return text.strip()




semantic_model = SentenceTransformer("all-MiniLM-L6-v2")

def semantic_similarity(text1, text2):
    emb1 = semantic_model.encode(text1, convert_to_tensor=True)
    emb2 = semantic_model.encode(text2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2).item()
    return round(float(score), 4)

def compare_controls(framework_df, client_list):
    results = []

    for client in client_list:
        c_name = client["Client_Control_Name"]
        c_desc = client["Client_Control_Description"]

        best_match = None
        best_score = -1

        # ---------- Search best framework match ----------
        for _, row in framework_df.iterrows():

            f_id   = row.get("Control_id", "")
            f_name = row.get("Control_name", "")
            f_desc = row.get("Control_description", "")

            deployment_pts =row.get("Deployment_points", "")

            # FULL comparison text
            framework_text = f"{f_name} {f_desc} {deployment_pts}"
            client_text    = f"{c_name} {c_desc}"

            score = semantic_similarity(client_text, framework_text)

            if score > best_score:
                best_score = score
                best_match = row

        # ---------- Add only the BEST match result ----------
        results.append({
            "Client_Control_Name": c_name,
            "Client_Control_Description": c_desc,
            "Framework_Control_Id": best_match.get("Control_id", ""),
            "Framework_Control_Name": best_match.get("Control_name", ""),
            "Framework_Control_Description": best_match.get("Control_description", ""),
            "Deployment_Points": best_match.get("Deployment_points", []),
            "Comparison_Score": best_score
        })

    return pd.DataFrame(results)
#--------------------------------
# Creating chunks of the text
#--------------------------------
def chunk_text(text_list, chunk_size=1000):
    chunks = []
    current = ""

    for line in text_list:
        if len(current) + len(line) <= chunk_size:
            current += " " + line
        else:
            chunks.append(current.strip())
            current = line

    if current:
        chunks.append(current.strip())

    return chunks


# --------------------------
# 3. Insert chunks into vector DB
# --------------------------
def insert_into_chroma(collection, chunks):
    ids = []
    for i, chunk in enumerate(chunks):
        ids.append(str(i))
    collection.add(
        documents=chunks,
        ids=ids
    )

# --------------------------
# NEW: Retrieve similar chunks (RAG Retrieval)
# --------------------------
def retrieve_similar_chunks(collection, query, top_k=70):
    """
    Return clean list of chunk strings.
    """
    result = collection.query(
        query_texts=[query],
        n_results=top_k
    )

    try:
        docs = result["documents"][0]
        return [d for d in docs if d.strip()]
    except:
        return []


# --------------------------
# 4. RAG QUERY: Ask LLM to Extract Controls
# --------------------------
client = OpenAI(api_key=os.environ['openai_key'])

import json
# --------------------------
# 4. EXTRACT CONTROL: Ask LLM to Extract Controls  (modified to use a list of strings)
# --------------------------
REGEX = r"\b(?:[A-Z]+(?:\.[A-Z]+)*[-.]?)?\d+(?:\.\d+)*\b"
def extract_controls_full(chunks,uuid_of_file):
    global status_msg

    '''
    # 1. Retrieve top relevant chunks
    rag_chunks = retrieve_similar_chunks(query, top_k=70)

    print("Retrieved chunks = ", len(rag_chunks))
    for i, c in enumerate(rag_chunks):
        print(f"\n--- Chunk {i+1} ---\n{c[:400]}\n")


    # If RAG fails, fallback to all chunks
    if not rag_chunks:
        rag_chunks = chunks
    #The following variable is used in the prompt (for extraction of the controls)
    #This way relevant chunks are used for this purpose

    #rag_chunks = chunks
    combined_context = "\n\n---\n\n".join(rag_chunks)
    '''
    prompt = f"""
You are a strict JSON generator.
Extract ALL compliance controls from the following text.
Do NOT skip ANY control.
Do NOT give anything extra.
Strictly search in the document.

STRICTLY extract all control IDs using the regex ONLY.

Do NOT invent or skip any ID.

STRICTLY extract all control IDs using this regex:
{REGEX!r}

Treat ALL numeric headings (0.1,0.2,0.3,0.4,1.1,1.2,2.3,1.1.2,A.1.2,A.2 etc.) as controls.
Do NOT treat them as TOC headings.

EXTRACT ONLY if all three are present in this sequence- ID, NAME and DESCRIPTION otherwise return null.

DESCRIPTION CAN BE OF 5 TO 1500 characters.

Consider all the controls given till the end.

Use JSON list ONLY:

[
  {{
    "Control_id": "",
    "Control_name": "",
    "Control_type":"",
    "Control_description": ""
  }}
]

TEXT:
{chunks}

Return ONLY JSON. No markdown. No text outside JSON.
"""

    print("extracting controls")
    status_msg[uuid_of_file] = "extracting controls from document."
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    status_msg[uuid_of_file] = "extracting deployment points."

    prompt = f"""
You are an anlyser/interpreter.
From each of the given compliance controls in the given JSON, analyse and retrieve the deployment points.
Do NOT skip ANY control.


INTERPRET THE TEXT IN THE DESCRIPTION AND GENERATE 5 to 6 POINTS FOR THE DEPLOYMENT (for each control seperatrly).
STORE THE DEPLOYMENT related points under Deployment_points heading.
Deployment points must describe:
- how this control is implemented,
- what actions/steps are required,
- how to operationalize the control.
- any other important point of deployment.
Every point should be numbered.
All the points should be stored in the form of a single string.

Use the following JSON for the above operation:
{response}

After the above retrieval operation, add one more heading in the JSON - Deployment_points and store the deployment points in it.
Use JSON list ONLY:

[
  {{
    "Control_id": "",
    "Control_name": "",
    "Control_type":"",
    "Control_description": "",
    "Deployment_points": ""
  }}
]

TEXT:
{chunks}

Return ONLY JSON. No markdown. No text outside JSON.
"""

    print("extracting deployment points.")
    status_msg[uuid_of_file] = "extracting deployment points"

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    try:
        return json.loads(response.choices[0].message.content)
    except:
        print("⚠ JSON error")
        return []


def remove_duplicate_controls(controls):
    """
    Removes duplicate controls based on Control_id.
    Keeps only the FIRST occurrence and discards others.
    """
    unique = {}
    cleaned_list = []

    for ctrl in controls:
        cid = ctrl.get("Control_id", "").strip()

        # Skip invalid entries
        if cid == "":
            continue

        # Keep only the first time a Control_id appears
        if cid not in unique:
            unique[cid] = True
            cleaned_list.append(ctrl)

    return cleaned_list
# --------------------------
# 5. RUN PIPELINE
# --------------------------

# --------------------------
#Setup ChromaDB (Vector DB)
# --------------------------


def extract_controls(file_path,uuid_of_file):
    chroma_client = chromadb.PersistentClient(path="chroma_db/")

    # Always reset the collection so old chunks do not pollute results
    try:
        chroma_client.delete_collection("controls_collection")
    except:
        pass

    collection = chroma_client.create_collection(
        name="controls_collection",
        metadata={"hnsw:space": "cosine"}
    )

    # Step 1: Load document
    text_content_list = load_document(file_path)


    # Step 2: Chunk text
    chunks = chunk_text(text_content_list)

    # Step 3: Insert into vector DB
    insert_into_chroma(collection,chunks)
    #import time
    #time.sleep(15)
    # Step 4: Run extraction
    #controls = extract_controls_full(chunks)
    #query = "List all compliance control statements, IDs, Names, Types and Descriptions."
    out_path = os.path.join(output_folder,uuid_of_file+".json")
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    output_file = out_path

    # --- Check if JSON already exists ---
    if os.path.exists(output_file):
        print("JSON file already exists. Loading existing controls...")

        with open(output_file, "r") as f:
            json_text = f.read()

        controls = json.loads(json_text)

    else:
        print("Extracting controls...")

        controls = extract_controls_full(chunks,uuid_of_file)   

        # Convert to JSON text
        json_text = json.dumps(controls, indent=2)

        # Save JSON to file
        with open(output_file, "w") as f:
            f.write(json_text)

        print("JSON saved to:", output_file)

    print("done")


def extract_controls_user(file_path,uuid_of_file):
    chroma_client = chromadb.PersistentClient(path="chroma_db/")

    # Always reset the collection so old chunks do not pollute results
    try:
        chroma_client.delete_collection("controls_collection")
    except:
        pass

    collection = chroma_client.create_collection(
        name="controls_collection",
        metadata={"hnsw:space": "cosine"}
    )

    # Step 1: Load document
    text_content_list = load_document(file_path)


    # Step 2: Chunk text
    chunks = chunk_text(text_content_list)

    # Step 3: Insert into vector DB
    insert_into_chroma(collection,chunks)
    #import time
    #time.sleep(15)
    # Step 4: Run extraction
    #controls = extract_controls_full(chunks)
    #query = "List all compliance control statements, IDs, Names, Types and Descriptions."
    out_path = os.path.join(output_folder,uuid_of_file+".json")
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    output_file = out_path

    # --- Check if JSON already exists ---
    if os.path.exists(output_file):
        print("JSON file already exists. Loading existing controls...")

        with open(output_file, "r") as f:
            json_text = f.read()

        controls = json.loads(json_text)

    else:
        print("Extracting controls...")

        controls = extract_controls_only(chunks,uuid_of_file)   

        # Convert to JSON text
        json_text = json.dumps(controls, indent=2)

        # Save JSON to file
        with open(output_file, "w") as f:
            f.write(json_text)

        print("JSON saved to:", output_file)

    print("done")


def extract_controls_only(chunks,uuid_of_file):
    global status_msg

    '''
    # 1. Retrieve top relevant chunks
    rag_chunks = retrieve_similar_chunks(query, top_k=70)

    print("Retrieved chunks = ", len(rag_chunks))
    for i, c in enumerate(rag_chunks):
        print(f"\n--- Chunk {i+1} ---\n{c[:400]}\n")


    # If RAG fails, fallback to all chunks
    if not rag_chunks:
        rag_chunks = chunks
    #The following variable is used in the prompt (for extraction of the controls)
    #This way relevant chunks are used for this purpose

    #rag_chunks = chunks
    combined_context = "\n\n---\n\n".join(rag_chunks)
    '''
    prompt = f"""
You are a strict JSON generator.
Extract ALL compliance controls from the following text.
Do NOT skip ANY control.
Do NOT give anything extra.
Strictly search in the document.

STRICTLY extract all control IDs using the regex ONLY.

Do NOT invent or skip any ID.

STRICTLY extract all control IDs using this regex:
{REGEX!r}

Treat ALL numeric headings (0.1,0.2,0.3,0.4,1.1,1.2,2.3,1.1.2,A.1.2,A.2 etc.) as controls.
Do NOT treat them as TOC headings.

EXTRACT ONLY if all three are present in this sequence- ID, NAME and DESCRIPTION otherwise return null.

DESCRIPTION CAN BE OF 5 TO 1500 characters.

Consider all the controls given till the end.

Use JSON list ONLY:

[
  {{
    "Control_id": "",
    "Control_name": "",
    "Control_type":"",
    "Control_description": ""
  }}
]

TEXT:
{chunks}

Return ONLY JSON. No markdown. No text outside JSON.
"""

    print("extracting controls")
    status_msg[uuid_of_file] = "extracting controls from pdf"
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    status_msg[uuid_of_file] = "extraction complete"
 
    try:
        return json.loads(response.choices[0].message.content)
    except:
        print("⚠ JSON error")
        return []

    

/home/jiten/rishit/api_new/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

In [11]:
client_df = pd.DataFrame(pd.read_json('test_2_user.json'))

In [4]:
expert_df = pd.DataFrame(pd.read_json('test_1_expert.json'))

In [12]:
client_df.head()

,Control_id,Control_name,Control_type,Control_description
0,PO.1,Define Security Requirements for Software Deve...,,Ensure that security requirements for software...
1,PO.1.1,Identify and document all security requirements,,Identify and document all security requirement...
2,PO.1.2,Identify and document security requirements fo...,,Identify and document all security requirement...
3,PO.1.3,Communicate requirements to third parties,,Communicate requirements to all third parties ...
4,PO.2,Implement Roles and Responsibilities,,Ensure that everyone inside and outside of the...


In [30]:
def get_comparision_dict(user_filename, expert_filename):
    client_df = pd.DataFrame(pd.read_json(user_filename))
    expert_df = pd.DataFrame(pd.read_json(expert_filename))
    results = []
    for client_idx ,client_row in client_df.iterrows():
        c_name = client_row["Control_name"]
        c_desc = client_row["Control_description"]
        c_id = client_row['Control_id']
        best_score = -1
        best_match = None

        for row_idx, row in expert_df[['Control_id','Control_name', 'Control_description','Deployment_points']].iterrows():
            
            f_id   = row.get("Control_id", "")
            f_name = row.get("Control_name", "")
            f_desc = row.get("Control_description", "")

            deployment_pts =row.get("Deployment_points", "")
            framework_text = f"{f_name} {f_desc} {deployment_pts}"
            client_text    = f"{c_name} {c_desc}"

            score = semantic_similarity(client_text, framework_text)
            if score > best_score:
                best_score = score
                best_match = row

        
        results.append({
            "User_Document_Control_Name": c_name,
            "User_Document_Control_Description": c_desc,
            "User_Document_Control_Id":c_id,
            "Expert_Framework_Control_Id": best_match.get("Control_id", ""),
            "Expert_Framework_Control_Name": best_match.get("Control_name", ""),
            "Expert_Framework_Control_Description": best_match.get("Control_description", ""),
            "Deployment_Points": best_match.get("Deployment_points", []),
            "Comparison_Score": best_score,
            
        })
    results_df = pd.DataFrame(results)
    return results_df.to_dict(orient='records')

In [31]:
pd.DataFrame(get_comparision_dict('test_2_user.json','test_1_expert.json'))

,User_Document_Control_Name,User_Document_Control_Description,User_Document_Control_Id,Expert_Framework_Control_Id,Expert_Framework_Control_Name,Expert_Framework_Control_Description,Deployment_Points,Comparison_Score
0,Define Security Requirements for Software Deve...,Ensure that security requirements for software...,PO.1,PO.1,Define Security Requirements for Software Deve...,Ensure that security requirements for software...,1. Establish a centralized repository for secu...,0.9073
1,Identify and document all security requirements,Identify and document all security requirement...,PO.1.1,PO.1.2,Identify and document all security requirement...,Identify and document all security requirement...,1. Define a set of risk-based security require...,0.8459
2,Identify and document security requirements fo...,Identify and document all security requirement...,PO.1.2,PO.1.2,Identify and document all security requirement...,Identify and document all security requirement...,1. Define a set of risk-based security require...,0.8231
3,Communicate requirements to third parties,Communicate requirements to all third parties ...,PO.1.3,PO.1.3,Communicate requirements to all third parties,Communicate requirements to all third parties ...,1. Develop a core set of security requirements...,0.8437
4,Implement Roles and Responsibilities,Ensure that everyone inside and outside of the...,PO.2,PO.2,Implement Roles and Responsibilities,Ensure that everyone inside and outside of the...,1. Define clear roles and responsibilities for...,0.9197
5,Create new roles and alter responsibilities,Create new roles and alter responsibilities fo...,PO.2.1,PO.2.1,Create new roles and alter responsibilities fo...,Create new roles and alter responsibilities fo...,1. Assess the current roles and responsibiliti...,0.9274
6,Provide role-based training,Provide role-based training for all personnel ...,PO.2.2,PO.2.2,Provide role-based training for all personnel,Provide role-based training for all personnel ...,1. Identify the specific training needs for ea...,0.8520
